In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Bikes

**Please run all cells before this cell, including the previous example cells and the import cell at the top of the notebook.**

In [ ]:
trips = Table.read_table('trips_2.csv')
trips

Remember from last time that these bikes were used generally for 30 
minutes or less: 30*60 = 1800 seconds.  When we viewd the histogram without 
restricting the seconds on the x-axis, there were some outliers!

In [ ]:
commute = trips.where('Duration', are.below(1800))
commute.hist('Duration')

In [ ]:
commute.hist('Duration', bins=60, unit='second')

In [ ]:
# what is the area of this peak here?  What percentage of all 
# the trips in the data does this represent?  
(500-250)*.15

What is the most popular place to pick up a bike from?  Let's use group!

In [ ]:
starts = commute.group('Start Station').sort('count', descending=True)
starts

Can we see what the most common bike commutes are?  In other words, how many people take a bike from location A and leave that bike at location B?  Or how many people do the reverse commute?  We can use pivot to count the number of trips that originate at station and end at another.

In [ ]:
commute.pivot('Start Station', 'End Station')

In [ ]:
# take a look at the table again.  What does commute look like?
commute.show(3)

In [ ]:
# make a new table that only shows start station, end station, and duration
duration = commute.select(3, 6, 1)
duration

What is the minimum amount of time that a bike might be out for a specific trip? Did you know that we can use a list in our first argument for group so that we can group all trips that start and end at a specific station together?

In [ ]:
shortest = duration.group(['Start Station', 'End Station'], min).sort(2)
shortest

In [ ]:
#let's look at just one specific start station.  Can we tell geography by looking at duration?
from_cc = shortest.where('Start Station', are.containing('Civic Center BART')).sort(2)
from_cc

## Maps

**Please run all cells before this cell, including the previous example cells and the import cell at the top of the notebook.**

In [ ]:
stations = Table.read_table('station.csv')
stations

In [ ]:
Marker.map_table(stations.select('lat', 'long', 'name'))

In [ ]:
#what is the variable "landmark?"
stations.group('landmark')

In [ ]:
#Let's use a different color to identify each landmark
colors = stations.group('landmark').with_column(
    'color', make_array('blue', 'red', 'green', 'orange', 'purple'))
colors

In [ ]:
#If we add these color identifiers to our stations table
#we can view these in our map.
stations.join('landmark', colors)

In [ ]:
#Now let's take just what we need from this table to make a map that uses
#color to identify each station with it's "landmark"
map_table = stations.join('landmark', colors).select('lat', 'long', 'name', 'color')
Marker.map_table(map_table)

Let's restrict our view to just San Francisco by using .where 

In [ ]:
sf = stations.where('landmark', 'San Francisco')
Circle.map_table(sf.select('lat', 'long', 'name'), color='green')

Can we also use maps to visualize the most popular trips?

In [ ]:
#we had looked at the trips from the Civic Center with their duration
from_cc

In [ ]:
#we had also created a table called "starts" that looked at the most popular
#start stations
starts

In [ ]:
#If we join starts with the stations table, then we can add counts to our 
#location data
station_starts = stations.join('name', starts, 'Start Station')
station_starts

In [ ]:
#just select the columns we need here
station_starts.select('lat', 'long', 'name').with_columns(
    'color', 'blue',
    'area', station_starts.column('count') 
)

In [ ]:
#make a map where the size of the circle is determined by the numer of trips leaving that station
Circle.map_table(station_starts.select('lat', 'long', 'name').with_columns(
    'color', 'blue',
    'area', station_starts.column('count') 
))

## Combining Table Methods

In [ ]:
duration

In [ ]:
#the value of the average duration of a rental.
np.average(duration.column(2))

In [ ]:
# the name of the station where most rentals ended.
duration.group('End Station').sort('count', descending = True).column(0).item(0)

In [ ]:
duration.group('End Station')

In [ ]:
duration.group('End Station').sort('count', descending = True)

In [ ]:
duration.group('End Station').sort('count', descending = True).column(0).item(0)

In [ ]:
duration.group('End Station', np.average)

In [ ]:
duration.group('End Station', np.average).where(2, are.above(10*60))

In [ ]:
# number of stations for which the average duration ending at that station was
# more than 10 minutes. 
duration.group('End Station', np.average).where(2, are.above(10*60)).num_rows

## Drinks - More Table Review

In [ ]:
drinks = Table(['Drink', 'Cafe', 'Price']).with_rows([
    ['Milk Tea', 'Tea One', 4],
    ['Espresso', 'Nefeli',  2],
    ['Coffee',    'Nefeli', 3],
    ['Espresso', "Abe's",   2]
])
drinks

In [ ]:
discounts = Table().with_columns(
    'Coupon % off', make_array(5, 50, 25),
    'Location', make_array('Tea One', 'Nefeli', 'Tea One')
)
discounts

In [ ]:
a = drinks.join('Cafe', discounts, 'Location')
a

In [ ]:
a = a.with_column("Disc", a.column('Price')*(1- a.column(3)/100))
a

In [ ]:
a = a.drop(2,3)
a

In [ ]:
a.group('Cafe', min)

In [ ]:
a.sort('Disc').sort('Cafe', distinct = True)

## Comparisons and Booleans

In [ ]:
1 > 2

In [ ]:
3 > 2

In [ ]:
np.arange(5) == 2

Closer look at .where

In [ ]:
pop = ('http://www2.census.gov/programs-surveys/popest/datasets/2010-2015/national/asrh/nc-est2015-agesex-res.csv')
full_census_table = Table.read_table(pop)
partial = full_census_table.select('SEX', 'AGE', 'POPESTIMATE2010', 'POPESTIMATE2015')
us_pop = partial.relabeled(2, '2010').relabeled(3, '2015')
us_pop

In [ ]:
us_pop.column('2015') / us_pop.column('2010') 

In [ ]:
#For whom was here more than 
us_pop.where(us_pop.column('2015') / us_pop.column('2010') > 1.5)

In [ ]:
# Let's do one more table with advanced where
duration

In [ ]:
# The average duration of trips that started and ended at the same station
np.average(duration.column(2))

In [ ]:
duration.where(duration.column(0) == duration.column(1))

In [ ]:
# The average duration of trips that started and ended at the same station
np.average(duration.where(duration.column(0) == duration.column(1)).column(2))

In [ ]:
# The average duration of trips that started and ended at different stations
np.average(duration.where(duration.column(0) != duration.column(1)).column(2))

## Comparison and bool values

In [ ]:
# comparing numbers 
x = 3
y = 4
x > y

In [ ]:
10/2 != y

In [ ]:
# What about strings?
'Canine' == 'Canne'

In [ ]:
# Let's try writing a function
def teen(x):
    return 13 <= x <= 19



In [ ]:
ages = Table().with_column('Age', np.arange(11, 50))
ages

In [ ]:
ages = Table().with_column('Age', np.arange(11, 50))
ages = ages.with_column('Teenager', ages.apply(teen, 'Age'))
ages


## Combining Bool Values

In [ ]:
x = 3
y = 4

In [ ]:
x < 2 or y > 5

In [ ]:
x < 2 or y > 3

In [ ]:
x < 2 and y > 3

Aggregating bool values

In [ ]:
True + True

In [ ]:
ages

In [ ]:
ages.column(1)

In [ ]:
sum(ages.column(1))

In [ ]:
sum(ages.column(1))/len(ages.column(1))

Predicates

In [ ]:
# create a predicate for "still young" 
still_young = are.between(35, 40)

In [ ]:
ages.where('Age', still_young)

In [ ]:
still_young(40)

In [ ]:
sum(ages.apply(still_young, 'Age'))

## Choice

In [ ]:
two_groups = make_array('treatment', 'control')
two_groups

In [ ]:
np.random.choice(two_groups)

In [ ]:
outcomes = np.random.choice(two_groups, 10)
outcomes

In [ ]:
outcomes == 'treatment'

In [ ]:
sum(outcomes == 'treatment')

In [ ]:
# count the number of true: 2 options

In [ ]:
# sum(outcomes)

In [ ]:
# np.count_nonzero(outcomes == ?)
np.count_nonzero(outcomes == 'treatment')